In [ ]:
# stacked generalization with linear meta model on blobs dataset
import tensorflow as tf

from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
import sklearn.linear_model as lm
logreg = lm.LogisticRegression(multi_class='multinomial', solver='lbfgs')
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from numpy import dstack

import os
from functools import partial
from collections import namedtuple
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPool2D, Activation, Dropout, BatchNormalization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD

In [ ]:
INPUT_ROOT = '/kaggle/input/lung-and-colon-cancer-histopathological-images'
DATA_DIR = os.path.join(INPUT_ROOT, 'lung_colon_image_set', 'lung_image_sets')

In [ ]:
data = image.ImageDataGenerator(validation_split = 0.2,)
BATCH_SIZE = 64
# Change batch size, if you run out of memory!
# Or increase it, if you have a lot of RAM (>> 16 Gb) for faster trainig
IMG_SHAPE = (256, 256, 3)

In [ ]:
generate_data = partial(
    data.flow_from_directory,
    DATA_DIR,
    class_mode = 'categorical',
    color_mode = 'rgb',
    target_size = IMG_SHAPE[:2],
    batch_size = BATCH_SIZE,
    shuffle = False,
    seed = 666
)

In [ ]:
print('Trainig data:')
train_iter = generate_data(subset='training')
print('Validating data:')
validate_iter = generate_data(subset='validation')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (256,256,3)) ,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="softmax"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(400,activation ="softmax"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(300,activation="softmax"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(200,activation ="softmax"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3,activation = "sigmoid")   #Adding the Output Layer
])



In [ ]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(learning_rate=0.01)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [ ]:
name = 'CustomCNN'

checkpointing = ModelCheckpoint(
    name,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

log_saving = CSVLogger(f'{name}-log.csv', append=True, separator=';')

In [ ]:
history = model.fit(train_iter,validation_data=validate_iter,steps_per_epoch=211,epochs=20,callbacks=[checkpointing, log_saving])